<a href="https://colab.research.google.com/github/saliq-2/FashionMnist/blob/main/resnetfashion_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [ ]:
# Import data (ignore the warnings!)
fashion_mnist = datasets.FashionMNIST('data/fashion', download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
print(fashion_mnist)
print(dir(fashion_mnist))
print(fashion_mnist.train_data)


Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data/fashion
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_check_exists', '_check_legacy_exist', '_format_transform_repr', '_is_protocol', '_load_data', '_load_legacy_data', '_repr_indent', 'class_to_idx', 'classes', 'data', 'download', 'extra_repr', 'mirrors', 'processed_folder', 'raw_folder', 'resources', 'root', 'target_transf

In [ ]:
def accuracy(true_labels, predicted_labels):
    """
    Compute the classification accuracy for given labels and predictions
    :param true_labels: list of gold labels
    :param predicted_labels: list of predicted labels of same length as true_labels
    :return: a float between 0 and 1 describing how accurate the predicted labels are
    """
    accuracy = 0.0

    # YOUR CODE HERE
    true_labels = torch.tensor(true_labels)
    predicted_labels = torch.tensor(predicted_labels)

    correct = (true_labels == predicted_labels).sum().item()
    total = len(true_labels)

    accuracy = correct / total
    #raise NotImplementedError
    return accuracy

In [ ]:
# Target classes
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'
}

In [ ]:
n_input = 784  # Fashion MNIST data input (img shape: 28*28)
n_classes = len(label_dict)  # Fashion MNIST total classes (0–9 digits)
n_samples = fashion_mnist.train_data.shape[0]  # Number of examples in training set
batch_size = 32
n_epochs = 10
summary_freq_batches = 10
learning_rate = 0.01
train_loader = torch.utils.data.DataLoader(fashion_mnist, batch_size = batch_size)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)

        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.downsample = downsample
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x

        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        if self.downsample:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

# ResNet (Mini)
class ResNetMini(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNetMini, self).__init__()
        self.in_channels = 16

        self.conv = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1, bias=False)  # 28x28 -> 28x28
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)

        self.layer1 = self._make_layer(block, 16, layers[0], stride=1)  # 28x28
        self.layer2 = self._make_layer(block, 32, layers[1], stride=2)  # 14x14
        self.layer3 = self._make_layer(block, 64, layers[2], stride=2)  # 7x7

        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))  # output: [batch_size, 64, 1, 1]
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride):
        downsample = None

        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = [block(self.in_channels, out_channels, stride, downsample)]
        self.in_channels = out_channels

        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.relu(self.bn(self.conv(x)))

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)

        x = self.avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [ ]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model initialization
model = ResNetMini(ResidualBlock, layers=[2, 2, 2], num_classes=10).to(device)

# Optimizer and loss function
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

# DataLoader
train_loader = torch.utils.data.DataLoader(fashion_mnist, batch_size=batch_size)

total_count = 0
for epoch_no in range(n_epochs):
    running_loss = 0.0
    model.train()  # Set model to training mode
    for batch_no, (batch_inputs, batch_labels) in enumerate(train_loader):
        total_count += batch_size

        # Move data to GPU
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        # Training step
        optimizer.zero_grad()
        outputs = model(batch_inputs)
        loss = loss_function(outputs, batch_labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    running_loss = running_loss / len(train_loader)
    print(f'Epoch {epoch_no} Training Loss--- {running_loss:.4f}')


Epoch 0 Training Loss--- 0.6503
Epoch 1 Training Loss--- 0.3650
Epoch 2 Training Loss--- 0.3002
Epoch 3 Training Loss--- 0.2617
Epoch 4 Training Loss--- 0.2335
Epoch 5 Training Loss--- 0.2100
Epoch 6 Training Loss--- 0.1908
Epoch 7 Training Loss--- 0.1725
Epoch 8 Training Loss--- 0.1567
Epoch 9 Training Loss--- 0.1415


In [ ]:
# Create a test DataLoader (batching avoids memory overload)
test_loader = torch.utils.data.DataLoader(fashion_mnist, batch_size=512, shuffle=False)

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch_inputs, batch_labels in test_loader:
        batch_inputs = batch_inputs.to(device)
        batch_labels = batch_labels.to(device)

        outputs = model(batch_inputs)
        _, preds = torch.max(outputs.data, 1)

        all_preds.append(preds.cpu())    # move to CPU for accuracy calc
        all_labels.append(batch_labels.cpu())

# Concatenate all predictions and labels
test_predictions = torch.cat(all_preds)
true_test_labels = torch.cat(all_labels)

# Compute accuracy
acc = accuracy(true_test_labels, test_predictions) * 100
print(f"\nTest accuracy after training of {total_count} iterations: {acc:.2f}%")



Test accuracy after training of 600000 iterations: 94.73%


/tmp/ipykernel_35/3259583533.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  true_labels = torch.tensor(true_labels)
/tmp/ipykernel_35/3259583533.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_labels = torch.tensor(predicted_labels)
